In [ ]:
#pip install scrapetube

In [ ]:
import scrapetube
import pandas as pd

In [ ]:
import os
import googleapiclient.discovery

# Set your YouTube API key as an environment variable
os.environ["YOUTUBE_API_KEY"] = "Your API Key"

# Initialize the YouTube Data API client
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=os.environ["YOUTUBE_API_KEY"])


In [ ]:
videos = pd.read_excel("English_channels_videos.xlsx")
videos.head()

In [ ]:
videos['Channel'].unique()

In [ ]:
channel_show_df = videos.groupby(['Channel','ShowName'],as_index=False).first()
channel_show_df = channel_show_df[['Channel', 'ShowName']]
channel_show_df
                           

In [ ]:
tags = ["Ayodhya", "section 370", "demonetisation", "farmer protest"]

In [ ]:
def search_video_in_show(channel_name, show_name, tag):
    # Search for videos in the specified channel and show with the given tag
    request = youtube.search().list(
        q=f"{channel_name} {show_name} {tag}",
        type="video",
        part="snippet",
        maxResults=1  # Retrieve only one result
    )
    
    # Print the exact request being made
    #print(f"Request URI: {request.uri}")
    
    # Execute the request
    video_search_response = request.execute()

    # Extract video information from the search results
    videos = video_search_response.get("items", [])

    return videos

In [ ]:
# Create an empty list to store the data
video_data = []

# Iterate through each row in the DataFrame
for index, row in channel_show_df.iterrows():
    channel_name = row["Channel"]
    show_name = row["ShowName"]

    # Iterate through each tag
    for tag in tags:
        # Search for a video in the specified channel and show with the given tag
        matching_videos = search_video_in_show(channel_name, show_name, tag)

        if matching_videos:
            video_id = matching_videos[0]["id"]["videoId"]
            video_title = matching_videos[0]["snippet"]["title"]

            # Append data to the list
            video_data.append({
                "Channel": channel_name,
                "ShowName": show_name,
                "Tag": tag,
                "Video ID": video_id,
                "Video Title": video_title
            })

# Create a DataFrame from the list of data
df = pd.DataFrame(video_data)

# Display the DataFrame
print(df)

In [ ]:
df.head()